## FishboneMoncriefID: An Einstein Toolkit Initial Data Thorn for Fishbone-Moncrief initial data

### NRPy+ Source Code for this module: [FishboneMoncriefID/FishboneMoncriefID.py](../edit/FishboneMoncriefID/FishboneMoncriefID.py) , which is fully documented in the [previous NRPy+ tutorial module](Tutorial-FishboneMoncriefID.ipynb) on using NRPy+ to construct these Fishbone-Moncrief initial data as SymPy expressions.


In this part of the tutorial, we will construct an Einstein Toolkit (ETK) thorn (module) that will set up Fishbone-Moncrief initial data. In a [previous tutorial module](Tutorial-FishboneMoncriefID.ipynb), we used NRPy+ to contruct the SymPy expressions for plane-wave initial data. 

We will construct this thorn in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the initial data into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

### Step 1: Call on NRPy+ to convert the SymPy expression for the Fishbone-Moncrief initial data into a C-code kernel. 

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. SymPy expressions for plane wave initial data are written inside [FishboneMoncriefID/FishboneMoncriefID.py](../edit/FishboneMoncriefID/FishboneMoncriefID.py), and we simply import them for use here.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")
# Force outputC::outCverbose to be false for this module to avoid gigantic C files
# filled with the non-CSE expressions for the Weyl scalars.
par.set_parval_from_str("outputC::outCverbose",False)

# Step 1c: Call the FishboneMoncriefID() function from within the
#          FishboneMoncriefID/FishboneMoncriefID.py module.
import FishboneMoncriefID.FishboneMoncriefID as fmid

# Step 2: Within the ETK, the 3D gridfunctions x, y, and z store the
#         Cartesian grid coordinates. Setting the gri.xx[] arrays
#         to point to these gridfunctions forces NRPy+ to treat
#         the Cartesian coordinate gridfunctions properly --
#         reading them from memory as needed.
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

# Step 3: Set up the Fishbone-Moncrief initial data. This sets all the ID gridfunctions.
fmid.FishboneMoncriefID()
Valencia3velocityU = ixp.register_gridfunctions_for_single_rank1("EVOL","Valencia3velocityU")

# -={ Spacetime quantities: Generate C code from expressions and output to file }=-
KerrSchild_to_print = [\
                     lhrh(lhs=gri.gfaccess("out_gfs","alpha"),rhs=fmid.IDalpha),\
                     lhrh(lhs=gri.gfaccess("out_gfs","betaU0"),rhs=fmid.IDbetaU[0]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","betaU1"),rhs=fmid.IDbetaU[1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","betaU2"),rhs=fmid.IDbetaU[2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","gammaDD00"),rhs=fmid.IDgammaDD[0][0]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","gammaDD01"),rhs=fmid.IDgammaDD[0][1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","gammaDD02"),rhs=fmid.IDgammaDD[0][2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","gammaDD11"),rhs=fmid.IDgammaDD[1][1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","gammaDD12"),rhs=fmid.IDgammaDD[1][2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","gammaDD22"),rhs=fmid.IDgammaDD[2][2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","KDD00"),rhs=fmid.IDKDD[0][0]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","KDD01"),rhs=fmid.IDKDD[0][1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","KDD02"),rhs=fmid.IDKDD[0][2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","KDD11"),rhs=fmid.IDKDD[1][1]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","KDD12"),rhs=fmid.IDKDD[1][2]),\
                     lhrh(lhs=gri.gfaccess("out_gfs","KDD22"),rhs=fmid.IDKDD[2][2]),\
                     ]
KerrSchild_CcodeKernel = fin.FD_outputC("returnstring",KerrSchild_to_print)

# -={ GRMHD quantities: Generate C code from expressions and output to file }=-
FMdisk_GRHD_hm1_to_print = [lhrh(lhs=gri.gfaccess("out_gfs","hm1"),rhs=fmid.hm1)]
FMdisk_GRHD_hm1_CcodeKernel = fin.FD_outputC("returnstring",FMdisk_GRHD_hm1_to_print)

FMdisk_GRHD_velocities_to_print = [\
                                 lhrh(lhs=gri.gfaccess("out_gfs","Valencia3velocityU0"),rhs=fmid.IDValencia3velocityU[0]),\
                                 lhrh(lhs=gri.gfaccess("out_gfs","Valencia3velocityU1"),rhs=fmid.IDValencia3velocityU[1]),\
                                 lhrh(lhs=gri.gfaccess("out_gfs","Valencia3velocityU2"),rhs=fmid.IDValencia3velocityU[2]),\
                                 ]
FMdisk_GRHD_velocities_CcodeKernel = fin.FD_outputC("returnstring",FMdisk_GRHD_velocities_to_print)

KerrSchild_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                              ["1","1","1"],["#pragma omp parallel for","",""],"",\
                              KerrSchild_CcodeKernel.replace("time","cctk_time"))

FMdisk_GRHD_velocities_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                          ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                          FMdisk_GRHD_velocities_CcodeKernel.replace("time","cctk_time"))

FMdisk_GRHD_hm1_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                   FMdisk_GRHD_hm1_CcodeKernel.replace("time","cctk_time"))

# Step 7: Create directories for the thorn if they don't exist.
!mkdir FishboneMoncriefID     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir FishboneMoncriefID/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 8: Write the C code kernel to file.
with open("FishboneMoncriefID/src/KerrSchild.h", "w") as file:
    file.write(str(KerrSchild_looped))
    
with open("FishboneMoncriefID/src/FMdisk_GRHD_velocities.h", "w") as file:
    file.write(str(FMdisk_GRHD_velocities_looped))
    
with open("FishboneMoncriefID/src/FMdisk_GRHD_hm1.h", "w") as file:
    file.write(str(FMdisk_GRHD_hm1_looped))


### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels.

We will write another C file with the functions we need here.

In [2]:
%%writefile FishboneMoncriefID/src/InitialData.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"

void FishboneMoncrief_KerrSchild(const cGH* restrict const cctkGH,const int *cctk_lsh,
                                 const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,
                                 CCTK_REAL *alphaGF,CCTK_REAL *betaU0GF,CCTK_REAL *betaU1GF,CCTK_REAL *betaU2GF,
                                 CCTK_REAL *gammaDD00GF,CCTK_REAL *gammaDD01GF,CCTK_REAL *gammaDD02GF,CCTK_REAL *gammaDD11GF,CCTK_REAL *gammaDD12GF,CCTK_REAL *gammaDD22GF,
                                 CCTK_REAL     *KDD00GF,CCTK_REAL     *KDD01GF,CCTK_REAL     *KDD02GF,CCTK_REAL     *KDD11GF,CCTK_REAL     *KDD12GF,CCTK_REAL     *KDD22GF)
{
  
  DECLARE_CCTK_PARAMETERS
  
#include "KerrSchild.h"
  
}

void FishboneMoncrief_FMdisk_GRHD_velocities(const cGH* restrict const cctkGH,const int *cctk_lsh,
                                             const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,
                                             CCTK_REAL *Valencia3velocityU0GF, CCTK_REAL *Valencia3velocityU1GF, CCTK_REAL *Valencia3velocityU2GF)
{
  
  DECLARE_CCTK_PARAMETERS
  
#include "FMdisk_GRHD_velocities.h"
  
}

void FishboneMoncrief_FMdisk_GRHD_hm1(const cGH* restrict const cctkGH,const int *cctk_lsh,
                                      const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,
                                      CCTK_REAL *hm1GF)
{
  
  DECLARE_CCTK_PARAMETERS
  
#include "FMdisk_GRHD_hm1.h"
  
}

void FishboneMoncrief_MainFunction(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS
  DECLARE_CCTK_PARAMETERS
  
  FishboneMoncrief_KerrSchild(cctkGH,cctk_lsh,
                              x,y,z,
                              alp,betax,betay,betaz,
                              gxx,gxy,gxz,gyy,gyz,gzz,
                              kxx,kxy,kxz,kyy,kyz,kzz);
  
  FishboneMoncrief_FMdisk_GRHD_velocities(cctkGH,cctk_lsh,
                                          x,y,z,
                                          Valencia3velocityU0,Valencia3velocityU1,Valencia3velocityU2);

  FishboneMoncrief_FMdisk_GRHD_hm1(cctkGH,cctk_lsh,
                                   x,y,z,
                                   hm1);
}

Overwriting FishboneMoncriefID/src/InitialData.c


### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. Specifically, this file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist and are declared "public" within those functions.

In [3]:
%%writefile FishboneMoncriefID/interface.ccl
implements: FishboneMoncriefID
inherits: admbase grid hydrobase


Overwriting FishboneMoncriefID/interface.ccl


2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [4]:
%%writefile FishboneMoncriefID/param.ccl
shares: grid

USES KEYWORD type

#["r_in","r_at_max_density","a","M"] A_b, kappa, gamma
restricted:
CCTK_REAL r_in "Fixes the inner edge of the disk"
{
 0.0:* :: "Must be positive"
} 6.0

restricted:
CCTK_REAL r_at_max_density "Radius at maximum disk density. Needs to be > r_in"
{
 0.0:* :: "Must be positive"
} 12.0

restricted:
CCTK_REAL a "The spin parameter of the black hole"
{
 -1.0:1.0 :: "Positive values, up to 1. Negative disallowed, as certain roots are chosen in the hydro fields setup. Check those before enabling negative spins!"
} 0.9375

restricted:
CCTK_REAL M "Kerr-Schild BH mass. Probably should always set M=1."
{
 0.0:* :: "Must be positive"
} 1.0

restricted:
CCTK_REAL A_b "Scaling factor for the vector potential"
{
 *:* :: ""
} 1.0

restricted:
CCTK_REAL kappa "Equation of state: P = kappa * rho^gamma"
{
 0.0:* :: "Positive values"
} 1.0e-3

restricted:
CCTK_REAL gamma "Equation of state: P = kappa * rho^gamma"
{
 0.0:* :: "Positive values"
} 1.3333333333333333333333333333


Overwriting FishboneMoncriefID/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We specify here the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [5]:
%%writefile FishboneMoncriefID/schedule.ccl

schedule FishboneMoncrief_MainFunction at CCTK_INITIAL as FishboneMoncrief_InitialData
{
  STORAGE:       FishboneMoncriefID::FMdisk_GRHD_velocities_group[3]
  STORAGE:       FishboneMoncriefID::FMdisk_GRHD_hm1_group[3]
  LANG:          C
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::y(Everywhere)
  WRITES: admbase::alp(Everywhere)
  WRITES: admbase::betax(Everywhere)
  WRITES: admbase::betay(Everywhere)
  WRITES: admbase::betaz(Everywhere)
  WRITES: admbase::kxx(Everywhere)
  WRITES: admbase::kxy(Everywhere)
  WRITES: admbase::kxz(Everywhere)
  WRITES: admbase::kyy(Everywhere)
  WRITES: admbase::kyz(Everywhere)
  WRITES: admbase::kzz(Everywhere)
  WRITES: admbase::gxx(Everywhere)
  WRITES: admbase::gxy(Everywhere)
  WRITES: admbase::gxz(Everywhere)
  WRITES: admbase::gyy(Everywhere)
  WRITES: admbase::gyz(Everywhere)
  WRITES: admbase::gzz(Everywhere)
  WRITES: Valencia3velocity0(Everywhere)
  WRITES: Valencia3velocity1(Everywhere)
  WRITES: Valencia3velocity2(Everywhere)
  WRITES: hm1(Everywhere)
} "Initial data for Fishbone-Moncrief disks"


Overwriting FishboneMoncriefID/schedule.ccl


#### Step 2c: Add the C code to the Einstein Toolkit compilation list.

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [6]:
%%writefile FishboneMoncriefID/src/make.code.defn
SRCS = InitialData.c

Overwriting FishboneMoncriefID/src/make.code.defn
